<a href="https://colab.research.google.com/github/BRV12G/Final_year_Project/blob/main/logistic_fitness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, mean_squared_error
import numpy as np

# Load the dataset
file_path = '/content/cleaned_diet_split_exercises.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Preprocess the data
# Handling missing values
data['Equipment3'] = data['Equipment3'].fillna('None')  # Replace missing equipment with 'None'
data = data.dropna(subset=['d_vegetables', 'd_juice', 'd_proteinintake'])  # Drop rows with missing diet data

# Encode categorical variables
categorical_columns = [
    'Gender', 'Hypertension', 'Diabetes', 'BMI Class', 'Fitness Goal', 'Fitness Type',
    'Exercise1', 'Exercise2', 'Exercise3', 'Equipment1', 'Equipment2', 'Equipment3'
]
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    data[column] = data[column].str.strip()  # Strip whitespace
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Scale numerical features
numerical_columns = ['Age', 'Height', 'Weight', 'BMI']
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# Define features and targets
X = data.drop(columns=[
    'ID', 'Exercise1', 'Exercise2', 'Exercise3', 'Duration of Workout', 'Calories Burnt',
    'Water Intake (Litres)', 'Equipment1', 'Equipment2', 'Equipment3',
    'd_vegetables', 'd_juice', 'd_proteinintake'
])
y_classification = data[['Exercise1', 'Exercise2', 'Exercise3', 'Equipment1', 'Equipment2', 'Equipment3']]
y_regression = data[['Duration of Workout', 'Calories Burnt', 'Water Intake (Litres)']]
y_diet = data[['d_vegetables', 'd_juice', 'd_proteinintake']]

# Split data into train and test sets
X_train, X_test, y_train_class, y_test_class = train_test_split(X, y_classification, test_size=0.2, random_state=42)
_, _, y_train_reg, y_test_reg = train_test_split(X, y_regression, test_size=0.2, random_state=42)
_, _, y_train_diet, y_test_diet = train_test_split(X, y_diet, test_size=0.2, random_state=42)

# Train Logistic Regression models for classification tasks
classification_models = {}
for output in y_classification.columns:
    model = LogisticRegression(max_iter=500, random_state=42)
    model.fit(X_train, y_train_class[output])
    classification_models[output] = model

# Train Random Forest models for regression tasks
regression_models = {}
for output in y_regression.columns:
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train_reg[output])
    regression_models[output] = model

# Function to preprocess a single input record
def preprocess_input(record):
    # Convert input to DataFrame
    input_data = pd.DataFrame([record], columns=X.columns)

    # Encode categorical inputs
    for column in categorical_columns:
        if column in input_data:
            try:
                input_data[column] = label_encoders[column].transform(input_data[column].str.strip())
            except ValueError:
                print(f"Unseen label in column '{column}': {input_data[column].iloc[0]}")
                input_data[column] = -1  # Handle unseen labels with a default value (e.g., -1)

    # Scale numerical inputs
    input_data[numerical_columns] = scaler.transform(input_data[numerical_columns])

    return input_data

# Function to generate recommendations for a single input
def generate_recommendations(record):
    processed_input = preprocess_input(record)
    recommendations = {}

    # Classification predictions
    for output, model in classification_models.items():
        pred = model.predict(processed_input)[0]
        pred = label_encoders[output].inverse_transform([pred])[0]  # Decode prediction
        recommendations[output] = pred

    # Regression predictions
    for output, model in regression_models.items():
        pred = model.predict(processed_input)[0]
        recommendations[output] = round(pred, 2)  # Round to 2 decimal places

    # Diet recommendations
    for output in y_diet.columns:
        try:
            pred = label_encoders[output].inverse_transform([classification_models[output].predict(processed_input)[0]])[0]
            recommendations[output] = pred
        except KeyError:
            recommendations[output] = "No recommendation available"

    return recommendations

# Interactive Input
def get_user_input():
    print("Enter the following details:")
    user_input = {
        'Gender': input("Gender (Male/Female): ").strip(),
        'Age': int(input("Age: ")),
        'Height': int(input("Height (in cm): ")),
        'Weight': float(input("Weight (in kg): ")),
        'Hypertension': input("Hypertension (Yes/No): ").strip(),
        'Diabetes': input("Diabetes (Yes/No): ").strip(),
        'BMI': float(input("BMI: ")),
        'BMI Class': input("BMI Class (Underweight/Normal/Overweight/Obese): ").strip(),
        'Fitness Goal': input("Fitness Goal (e.g., Weight Loss/Maintain Fitness): ").strip(),
        'Fitness Type': input("Fitness Type (e.g., Cardio/Strength): ").strip()
    }
    return user_input

# Get user input
user_input = get_user_input()

# Generate recommendations
recommendations = generate_recommendations(user_input)

# Display recommendations
print("\nRecommended Plan:")
for key, value in recommendations.items():
    print(f"{key}: {value}")


<ipython-input-1-85e880ca1450>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data[column].str.strip()  # Strip whitespace
<ipython-input-1-85e880ca1450>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = le.fit_transform(data[column])
<ipython-input-1-85e880ca1450>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

Enter the following details:
Gender (Male/Female): Male 
Age: 23
Height (in cm): 160
Weight (in kg): 70
Hypertension (Yes/No): Yes
Diabetes (Yes/No): No
BMI: 27.34
BMI Class (Underweight/Normal/Overweight/Obese): Overweight
Fitness Goal (e.g., Weight Loss/Maintain Fitness): Weight Loss 
Fitness Type (e.g., Cardio/Strength): High Interval Training 
Unseen label in column 'Fitness Type': High Interval Training

Recommended Plan:
Exercise1: Squats
Exercise2: Deadlifts
Exercise3: Bench Presses
Equipment1: Jump Rope
Equipment2: Medicine Ball
Equipment3: None
Duration of Workout: 78.95
Calories Burnt: 629.13
Water Intake (Litres): 3.63
d_vegetables: No recommendation available
d_juice: No recommendation available
d_proteinintake: No recommendation available
